In [ ]:
suppressMessages(suppressWarnings(library(Seurat)))
suppressMessages(suppressWarnings(library(tidyverse)))
suppressMessages(suppressWarnings(library(ggpubr)))
suppressMessages(suppressWarnings(library(edgeR)))
suppressMessages(suppressWarnings(library(DESeq2)))
suppressMessages(suppressWarnings(library(gprofiler2)))
suppressMessages(suppressWarnings(library(reshape2)))
suppressMessages(suppressWarnings(library(doMC)))

In [ ]:
glia <- readRDS("/projects/perslab/people/jmg776/projects/DVC/output/Seurat_objs/integrated/glia_finalized_2025.rds")
neurons <- readRDS("/projects/perslab/people/jmg776/projects/DVC/output/Seurat_objs/integrated/neurons_finalized_2025.rds")
neurons$cell.type <- gsub("_(\\d+)_", "\\1.", neurons$cell.type)

In [ ]:
compute.DESeq2.stats <- function(seurat.obj, species, year, group1, group2) {
  
  species.year <- paste0(species, ".", year)
  species.assay <- paste0(species, "_", "RNA")
  groups <- c(group2, group1)
  
  # Define cell types
  cell.types <- na.omit(unique(seurat.obj$cell.type[which(seurat.obj$species == species & 
                                                    seurat.obj$year == year)]))
  
  # Loop through cell types
  DEGs.DESeq2 <- foreach(i = cell.types) %dopar% {
    
    seurat.obj.sub <- subset(seurat.obj, cell.type == i & dataset == species.year & 
                               treatment %in% groups)
  
    samples.keep <- names(table(seurat.obj.sub$hash.ID))[which(table(seurat.obj.sub$hash.ID) >= 3)]
    print(samples.keep)
  
    # Skip if no samples constitute at least 3 cells
    statement1 <- length(samples.keep) == 0 
    # Skip if both treatment groups are not found
    statement2 <- length(unique(gsub("\\-[0-9]+", "", samples.keep))) != 2 
    # Skip if at least two samples in each treatment group are not found
    statement3 <- min(table(gsub("\\-[0-9]+", "", samples.keep))) < 2

    print(statement1)
    print(statement2)
    print(statement3)
      
    if (statement1 | statement2 | statement3) {
      print(NA)
      } else {

    
    seurat.obj.sub <- subset(seurat.obj.sub, hash.ID %in% samples.keep)
    
    genes.keep <- which(apply(seurat.obj.sub@assays[[species.assay]]@counts, 1,
                               function(x) {sum(x != 0)}) >= (ncol(seurat.obj.sub)/10))

    seurat.obj.sub <- subset(seurat.obj.sub, features = names(genes.keep))

    # Pseudobulk
    sample <- unique(seurat.obj.sub$hash.ID)
    genes <- rownames(seurat.obj.sub@assays[[species.assay]]@counts)

    pseudo.data <- data.frame(matrix(NA, nrow = length(genes), ncol = length(sample)))
    rownames(pseudo.data) <- genes
    colnames(pseudo.data) <- sample
    
    pseudo.meta <- data.frame(matrix(NA, nrow = length(sample), ncol = 2))
    colnames(pseudo.meta) <- c("sample", "treatment")
    
    for (j in 1:length(sample)) {
      
      pseudo.data[, j] <-
        as.integer(apply(seurat.obj.sub@assays[[species.assay]]@counts[genes, which(seurat.obj.sub$hash.ID == sample[j]), drop = F], 1, sum))
      pseudo.meta$sample[j] <- sample[j]
      pseudo.meta$treatment[j] <- 
        unique(as.character(seurat.obj.sub$treatment[which(seurat.obj.sub$hash.ID == sample[j])]))
      }
  
   # Normalization
   condition <- factor(gsub("-[0-9]+", "", colnames(pseudo.data)), levels = groups)
   design <- data.frame(condition = condition)
   pr

   dds <- DESeqDataSetFromMatrix(pseudo.data, DataFrame(design), ~ condition)
   dds <- DESeq(dds)
   resultsNames(dds)
   DEGs <- results(dds, name=resultsNames(dds)[2]) 
   DEGs <- data.frame(DEGs)
  
   DEGs 
      }
   }
   names(DEGs.DESeq2) <- cell.types
   DEGs.DESeq2 <- DEGs.DESeq2[!(is.na(DEGs.DESeq2))]

   return(DEGs.DESeq2)
}

# Neurons

In [ ]:
# Rat neurons
DEGs.rat.neurons.acute.A8 <- compute.DESeq2.stats(seurat.obj = neurons, species = "rat", year = "2023", group1 = "A8-A", group2 = "V-A")
saveRDS(DEGs.rat.neurons.acute.A8, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_rat_neurons_acute_AM833_2025.rds")

DEGs.rat.neurons.chronic.A8 <- compute.DESeq2.stats(seurat.obj = neurons, species = "rat", year = "2023", group1 = "A8-C", group2 = "WM-C")
saveRDS(DEGs.rat.neurons.chronic.A8, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_rat_neurons_chronic_AM833_2025.rds")

DEGs.rat.neurons.WMlean <- compute.DESeq2.stats(seurat.obj = neurons, species = "rat", year = "2023", group1 = "L-A", group2 = "WM-C")
saveRDS(DEGs.rat.neurons.WMlean, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_rat_neurons_WMlean_2025.rds")

DEGs.rat.neurons.lean <- compute.DESeq2.stats(seurat.obj = neurons, species = "rat", year = "2023", group1 = "L-A", group2 = "V-A")
saveRDS(DEGs.rat.neurons.lean, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_rat_neurons_lean_2025.rds")


# Mouse neurons
DEGs.mouse.neurons.acute.A8 <- compute.DESeq2.stats(seurat.obj = neurons, species = "mouse", year = "2023", group1 = "A8-A", group2 = "V-A")
saveRDS(DEGs.mouse.neurons.acute.A8, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_mouse_neurons_acute_AM833_2025.rds")

DEGs.mouse.neurons.chronic.A8 <- compute.DESeq2.stats(seurat.obj = neurons, species = "mouse", year = "2023", group1 = "A8-C", group2 = "WM-C")
saveRDS(DEGs.mouse.neurons.chronic.A8, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_mouse_neurons_chronic_AM833_2025.rds")

DEGs.mouse.neurons.acute.A1 <- compute.DESeq2.stats(seurat.obj = neurons, species = "mouse", year = "2023", group1 = "A1-A", group2 = "V-A")
saveRDS(DEGs.mouse.neurons.acute.A1, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_mouse_neurons_acute_AM1213_2025.rds")

DEGs.mouse.neurons.chronic.A1 <- compute.DESeq2.stats(seurat.obj = neurons, species = "mouse", year = "2023", group1 = "A1-C", group2 = "WM-C")
saveRDS(DEGs.mouse.neurons.chronic.A1, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_mouse_neurons_chronic_AM1213_2025.rds")

DEGs.mouse.neurons.WMlean <- compute.DESeq2.stats(seurat.obj = neurons, species = "mouse", year = "2023", group1 = "L-A", group2 = "WM-C")
saveRDS(DEGs.mouse.neurons.WMlean, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_mouse_neurons_WMlean_2025.rds")

DEGs.mouse.neurons.lean <- compute.DESeq2.stats(seurat.obj = neurons, species = "mouse", year = "2023", group1 = "L-A", group2 = "V-A")
saveRDS(DEGs.mouse.neurons.lean, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_mouse_neurons_lean_2025.rds")

# Glia

In [ ]:
# Rat glia
DEGs.rat.glia.acute.A8 <- compute.DESeq2.stats(seurat.obj = glia, species = "rat", year = "2023", group1 = "A8-A", group2 = "V-A")
saveRDS(DEGs.rat.glia.acute.A8, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_rat_glia_acute_AM833_2025.rds")

DEGs.rat.glia.chronic.A8 <- compute.DESeq2.stats(seurat.obj = glia, species = "rat", year = "2023", group1 = "A8-C", group2 = "WM-C")
saveRDS(DEGs.rat.glia.chronic.A8, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_rat_glia_chronic_AM833_2025.rds")

DEGs.rat.glia.WMlean <- compute.DESeq2.stats(seurat.obj = glia, species = "rat", year = "2023", group1 = "L-A", group2 = "WM-C")
saveRDS(DEGs.rat.glia.WMlean, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_rat_glia_WMlean_2025.rds")

DEGs.rat.glia.lean <- compute.DESeq2.stats(seurat.obj = glia, species = "rat", year = "2023", group1 = "L-A", group2 = "V-A")
saveRDS(DEGs.rat.glia.lean, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_rat_glia_lean_2025.rds")


# Mouse glia
DEGs.mouse.glia.acute.A8 <- compute.DESeq2.stats(seurat.obj = glia, species = "mouse", year = "2023", group1 = "A8-A", group2 = "V-A")
saveRDS(DEGs.mouse.glia.acute.A8, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_mouse_glia_acute_AM833_2025.rds")

DEGs.mouse.glia.chronic.A8 <- compute.DESeq2.stats(seurat.obj = glia, species = "mouse", year = "2023", group1 = "A8-C", group2 = "WM-C")
saveRDS(DEGs.mouse.glia.chronic.A8, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_mouse_glia_chronic_AM833_2025.rds")

DEGs.mouse.glia.acute.A1 <- compute.DESeq2.stats(seurat.obj = glia, species = "mouse", year = "2023", group1 = "A1-A", group2 = "V-A")
saveRDS(DEGs.mouse.glia.acute.A1, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_mouse_glia_acute_AM1213_2025.rds")

DEGs.mouse.glia.chronic.A1 <- compute.DESeq2.stats(seurat.obj = glia, species = "mouse", year = "2023", group1 = "A1-C", group2 = "WM-C")
saveRDS(DEGs.mouse.glia.chronic.A1, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_mouse_glia_chronic_AM1213_2025.rds")

DEGs.mouse.glia.WMlean <- compute.DESeq2.stats(seurat.obj = glia, species = "mouse", year = "2023", group1 = "L-A", group2 = "WM-C")
saveRDS(DEGs.mouse.glia.WMlean, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_mouse_glia_WMlean_2025.rds")

DEGs.mouse.glia.lean <- compute.DESeq2.stats(seurat.obj = glia, species = "mouse", year = "2023", group1 = "L-A", group2 = "V-A")
saveRDS(DEGs.mouse.glia.lean, file = "/projects/perslab/people/jmg776/projects/DVC/output/DEGs/single-cell/DESeq2_mouse_glia_lean_2025.rds")